## Imports

In [1]:
import sys
import os
import random
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import h5py
from h5py import File as HDF5File

import enum
print(enum.__file__)

/usr/lib/python3.8/enum.py


In [3]:
# Oh boy this might take a while
import tensorflow as tf
from tensorflow import keras

from keras.layers import Lambda, Input
from keras.layers import Dropout, Flatten, Dense
import keras.backend as K
from keras.models import Sequential, Model 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate

2024-01-15 19:23:43.717240: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 19:23:44.392360: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import yaml
with open("../config.yaml") as fin:
    config = yaml.safe_load(fin)
    data_dir = config["data_dir"]
    model_dir = config["model_dir"]
    output_dir = config["output_dir"]

In [5]:
task_name = "scalar1"

## Loading data

See `cnn/data.py`.

In [6]:
from cnn_data import load_data
X_train, Y_train, X_test, Y_test = load_data(task_name)

load_data from datafile /data/wifeng/photon-jet/data/h5/pi0_40-250GeV_100k.h5
load_data from datafile /data/wifeng/photon-jet/data/h5/gamma_40-250GeV_100k.h5
load_data from datafile /data/wifeng/photon-jet/data/h5/scalar1_40-250GeV_100k.h5
rng=Generator(PCG64), perm=[264512 162665 196504 ... 113500 283120 234089], n_train=21000, n_test=27900


In [7]:
print("Training shapes:")
print("  inputs:", X_train[0].shape, X_train[1].shape)
print("  labels:", Y_train.shape)

Training shapes:
  inputs: (21000, 4, 16, 1) (21000, 4, 128, 1)
  labels: (21000,)


## Build the model

In [8]:
visible1 = Input(shape=(4, 16, 1))
conv11 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)
print ('cov11.shape', conv11.shape)
print ('pool1.shape', pool11.shape)
print ('cov12.shape', conv12.shape)
print ('pool2.shape', pool12.shape)

cov11.shape (None, 4, 16, 32)
pool1.shape (None, 2, 8, 32)
cov12.shape (None, 2, 8, 16)
pool2.shape (None, 1, 4, 16)


2024-01-15 19:23:49.426260: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9804 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:88:00.0, compute capability: 7.5
2024-01-15 19:23:49.427235: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9804 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:89:00.0, compute capability: 7.5
2024-01-15 19:23:49.428142: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9804 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:b1:00.0, compute capability: 7.5
2024-01-15 19:23:49.428972: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 9804 MB memory:  -> device: 3, name: NVIDIA GeForce RTX

In [9]:
visible2 = Input(shape=(4, 128, 1))
conv21 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)
print ('cov21.shape', conv21.shape)
print ('poo21.shape', pool21.shape)
print ('cov22.shape', conv22.shape)
print ('poo22.shape', pool22.shape)

cov21.shape (None, 4, 128, 32)
poo21.shape (None, 2, 64, 32)
cov22.shape (None, 2, 64, 16)
poo22.shape (None, 1, 32, 16)


In [10]:
merge = concatenate([flat1, flat2])

In [32]:
# interpretation model
hidden1 = Dense(50, activation='relu')(merge)
dropout1 = Dropout(0.1)(hidden1)
hidden2 = Dense(20, activation='relu')(dropout1)
dropout2 = Dropout(0.1)(hidden2)
output = Dense(3)(dropout2)
cnn = Model(inputs=[visible1, visible2], outputs=output)

In [33]:
# summarize layers
# print(cnn.summary())

In [34]:
cnn.inputs

[<KerasTensor: shape=(None, 4, 16, 1) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 4, 128, 1) dtype=float32 (created by layer 'input_2')>]

## Train the model

In [35]:
def train_iteration(lr, epochs):
    cnn.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )    
    return cnn.fit(
        X_train, Y_train,
        epochs=epochs, batch_size=32,
        validation_data=(X_test, Y_test),
    )

In [36]:
train_iteration(lr=1e-2, epochs=5)

Epoch 1/5
657/657 [==============================] - 6s 6ms/step - loss: 0.5563 - accuracy: 0.8861 - val_loss: 0.1440 - val_accuracy: 0.9548
Epoch 2/5
657/657 [==============================] - 4s 7ms/step - loss: 0.2398 - accuracy: 0.9277 - val_loss: 0.2044 - val_accuracy: 0.9389
Epoch 3/5
657/657 [==============================] - 4s 5ms/step - loss: 0.2507 - accuracy: 0.9210 - val_loss: 0.2866 - val_accuracy: 0.9102
Epoch 4/5
657/657 [==============================] - 4s 5ms/step - loss: 0.2149 - accuracy: 0.9317 - val_loss: 0.1676 - val_accuracy: 0.9512
Epoch 5/5
657/657 [==============================] - 4s 6ms/step - loss: 0.3048 - accuracy: 0.9076 - val_loss: 0.2053 - val_accuracy: 0.9390


In [37]:
train_iteration(lr=2e-3, epochs=5)

Epoch 1/5
657/657 [==============================] - 7s 7ms/step - loss: 0.1550 - accuracy: 0.9573 - val_loss: 0.1267 - val_accuracy: 0.9655
Epoch 2/5
657/657 [==============================] - 4s 6ms/step - loss: 0.1797 - accuracy: 0.9540 - val_loss: 0.1510 - val_accuracy: 0.9606
Epoch 3/5
657/657 [==============================] - 4s 5ms/step - loss: 0.1425 - accuracy: 0.9599 - val_loss: 0.1137 - val_accuracy: 0.9677
Epoch 4/5
657/657 [==============================] - 3s 5ms/step - loss: 0.1386 - accuracy: 0.9603 - val_loss: 0.2088 - val_accuracy: 0.9686
Epoch 5/5
657/657 [==============================] - 4s 6ms/step - loss: 0.1349 - accuracy: 0.9600 - val_loss: 0.1582 - val_accuracy: 0.9689


In [38]:
train_iteration(lr=1e-3, epochs=5)

Epoch 1/5
657/657 [==============================] - 7s 7ms/step - loss: 0.1171 - accuracy: 0.9685 - val_loss: 0.1173 - val_accuracy: 0.9687
Epoch 2/5
657/657 [==============================] - 5s 7ms/step - loss: 0.1015 - accuracy: 0.9698 - val_loss: 0.1366 - val_accuracy: 0.9701
Epoch 3/5
657/657 [==============================] - 4s 5ms/step - loss: 0.1304 - accuracy: 0.9686 - val_loss: 0.1331 - val_accuracy: 0.9690
Epoch 4/5
657/657 [==============================] - 4s 6ms/step - loss: 0.1095 - accuracy: 0.9670 - val_loss: 0.1255 - val_accuracy: 0.9704
Epoch 5/5
657/657 [==============================] - 4s 6ms/step - loss: 0.1004 - accuracy: 0.9717 - val_loss: 0.1278 - val_accuracy: 0.9704


In [ ]:
train_iteration(lr=2e-4, epochs=5)

In [ ]:
train_iteration(lr=2e-4, epochs=10)

In [ ]:
# Save this model
os.makedirs(f"{model_dir}", exist_ok=True)

print(f"task name: {task_name}")
cnn.save(f"{model_dir}/{task_name}_cnn")

In [ ]:
import json

os.makedirs(f"{output_dir}/cnn", exist_ok=True)
with open(f"{output_dir}/cnn/{task_name}_train_history.json", "w") as fout:
    json.dump({
        key: history.history[key] for key in ["loss", "val_loss", "acc", "val_acc"]
    }, fout)